In [1]:
import nltk
import re
import pandas as pd
from IPython.display import clear_output
from collections import defaultdict
from string import punctuation

#stopwords em portugues
from nltk.corpus import stopwords
stopwords = set(stopwords.words('portuguese') + list(punctuation) + ['“','”'])

#funçao pra limpeza de texto
from nltk.tokenize import TweetTokenizer
tweet_tokenizer = TweetTokenizer(strip_handles=True, reduce_len=True, preserve_case=False)

In [2]:
#funçao pra leitura de dados e criaçao de dataframe
def json_to_dataframe(nome_json, colunas, chunk):
    dados = pd.read_json(nome_json, convert_dates = False, lines = True, chunksize = chunk)    
    
    for df_tweets_retweets in dados:
        yield df_tweets_retweets[colunas]

In [3]:
#funçao pra limpar texto e retirar stopwords
def tokenizar_texto(texto_tweet):
    texto_tokenizado = tweet_tokenizer.tokenize(texto_tweet)

    if texto_tokenizado[0] == "rt" and texto_tokenizado[1] == ":":
        del texto_tokenizado[0]
    
    palavras_sem_stopwords = [palavra for palavra in texto_tokenizado if palavra not in stopwords] 
    return palavras_sem_stopwords

In [4]:
#funçao pra criar dicionario com a data e quantidade das palavras chaves
def contagem_pchaves_data(pchaves_lista, gerador):
    
    dict_cont_pchaves = defaultdict(int)
    dict_datas_pchaves = defaultdict(dict)
    contador = 0
    
    for dataframe in gerador:        
        for linha in dataframe.values:            
            texto_tweet = list(tokenizar_texto(linha[0]))

            for pchave in pchaves_lista:                
                if pchave in texto_tweet:
                    data = linha[1]['$date'][:10]
                    if data not in dict_datas_pchaves:
                        dict_cont_pchaves[pchave] = 0
                    dict_cont_pchaves[pchave] += 1
                    dict_datas_pchaves[data] = dict(dict_cont_pchaves)
            clear_output()
            contador += 1        
            print("%s tweets processados" %(contador))            
            print(dict(dict_datas_pchaves))
            
    escrever_arq_csv(dict_datas_pchaves)

In [9]:
#funçao pra escrever os resultados no arquivo csv
def escrever_arq_csv(dict_datas_pchaves):
    with open('aglomeracao_aglomeracoes_pchave100k.csv', 'w', encoding="utf-8") as f:
        f.write("Data;Ocorrencias 'Aglomeracao';Ocorrencias 'Aglomeracoes'\n")

        for key in dict_datas_pchaves.keys():
            f.write("%s;%s;%s\n"%(key, dict_datas_pchaves[key].get('aglomeração'), dict_datas_pchaves[key].get('aglomerações')))
            
    print('A contagem de palavras chaves por data foi salva com sucesso no arquivo')

In [ ]:
#execuçao do codigo
gerador = json_to_dataframe('100k_origin.json',["text","created_at"], 30)

contagem_pchaves_data(['aglomeração', 'aglomerações'], gerador)